In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot([1, 2, 3], [1, 4, 9])
plt.title("Тестовый график")
plt.show()

In [34]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt


class MNISTDataset(Dataset):
    def __init__(self, train=True, transform=None):
        super().__init__()
        self.dataset = torchvision.datasets.MNIST(
            root='./data', 
            train=train, 
            download=True, 
            transform=transform
        )
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx]


class CIFARDataset(Dataset):
    def __init__(self, train=True, transform=None):
        super().__init__()
        self.dataset = torchvision.datasets.CIFAR10(
            root='./data', 
            train=train, 
            download=True, 
            transform=transform
        )
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx]


def get_mnist_loaders(batch_size=64):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    train_dataset = MNISTDataset(train=True, transform=transform)
    test_dataset = MNISTDataset(train=False, transform=transform)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader


def get_cifar_loaders(batch_size=64):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])
    
    train_dataset = CIFARDataset(train=True, transform=transform)
    test_dataset = CIFARDataset(train=False, transform=transform)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader 

In [36]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class SimpleCNN(nn.Module):
    def __init__(self, input_channels=1, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, 32, 3, 1, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.25)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

#3x3 ядра
class SimpleCNN3x3(nn.Module):
    def __init__(self, input_channels=1, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(2304, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.25)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
# 5x5 ядра
class SimpleCNN5x5(nn.Module):
    def __init__(self, input_channels=1, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, 32, 5, padding=2)
        self.conv2 = nn.Conv2d(32, 64, 5, padding=2)
        self.conv3 = nn.Conv2d(64, 128, 5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(1152, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.25)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
#7x7 ядра
class SimpleCNN7x7(nn.Module):
    def __init__(self, input_channels=1, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, 16, 7, padding=3)
        self.conv2 = nn.Conv2d(16, 32, 7, padding=3)
        self.conv3 = nn.Conv2d(32, 64, 7, padding=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(576, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.25)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
#Комбинация разных размеров (1x1 + 3x3 + 5x5)
class SimpleCNNMixed(nn.Module):
    def __init__(self, input_channels=1, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, 32, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(1152, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.25)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
    
class CNNWithResidual(nn.Module):
    def __init__(self, input_channels=1, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, 32, 3, 1, 1)
        self.bn1 = nn.BatchNorm2d(32)
        
        self.res1 = ResidualBlock(32, 32)
        self.res2 = ResidualBlock(32, 64, 2)
        self.res3 = ResidualBlock(64, 64)
        
        self.pool = nn.AdaptiveAvgPool2d((4, 4))
        self.fc = nn.Linear(64 * 4 * 4, num_classes)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.res1(x)
        x = self.res2(x)
        x = self.res3(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


class CIFARCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1, 1)
        self.conv3 = nn.Conv2d(64, 128, 3, 1, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.25)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x 

In [111]:
class FullyConnectedModel(nn.Module):
    def __init__(self, config_path=None, input_size=None, num_classes=None, **kwargs):
        super().__init__()
        
        if config_path:
            self.config = self.load_config(config_path)
        else:
            self.config = kwargs

        self.input_size = input_size or self.config.get('input_size', 784)
        self.num_classes = num_classes or self.config.get('num_classes', 10)

        self.layers = self._build_layers()

    def load_config(self, config_path):
        """
        Загружает конфигурацию из JSON файла
        """
        with open(config_path, 'r') as f:
            return json.load(f)

    def _build_layers(self):
        layers = []
        prev_size = self.input_size

        layer_config = self.config.get('layers', [])

        for layer_spec in layer_config:
            layer_type = layer_spec['type']

            match layer_type:
                case 'linear':
                    out_size = layer_spec['size']
                    layers.append(nn.Linear(prev_size, out_size))
                    prev_size = out_size

                case 'relu':
                    layers.append(nn.ReLU())

                case 'sigmoid':
                    layers.append(nn.Sigmoid())

                case 'tanh':
                    layers.append(nn.Tanh())

                case 'dropout':
                    rate = layer_spec.get('rate', 0.5)
                    layers.append(nn.Dropout(rate))

                case 'batch_norm':
                    layers.append(nn.BatchNorm1d(prev_size))

                case 'layer_norm':
                    layers.append(nn.LayerNorm(prev_size))

                case _:
                    raise ValueError(f"Неизвестный тип слоя: {layer_type}")

        # Всегда добавляем финальный слой для классификации
        layers.append(nn.Linear(prev_size, self.num_classes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # "выпрямляем" картинку в вектор
        return self.layers(x)

# 3.1
class CustomConv2d(nn.Module):
    # Добавил возможность применения аддитивного шума к входным данным внутри слоя
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, noise_std=0.1):
        super(CustomConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.noise_std = noise_std

    def forward(self, x):
        if self.training:
            noise = torch.randn_like(x) * self.noise_std
            x = x + noise
        return self.conv(x)
    
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv(x)
        return self.sigmoid(x)
    
class CustomReLU(nn.Module):
    def __init__(self):
        super(CustomReLU, self).__init__()

    def forward(self, x):
        self.input = x
        return x.clamp(min=0)

    def backward(self, grad_output):
        # Обычный ReLU: градиент пропускается, если input > 0
        grad_input = grad_output.clone()
        grad_input[self.input < 0] = 0
        return grad_input
    
class CustomAdaptiveAvgPool2d(nn.Module):
    def __init__(self, output_size):
        super(CustomAdaptiveAvgPool2d, self).__init__()
        self.output_size = output_size

    def forward(self, x):
        bs, c, h, w = x.size()
        oh, ow = self.output_size
        sh, sw = h // oh, w // ow
        x = x.view(bs, c, oh, sh, ow, sw)
        x = x.mean([3, 5])  # усреднение по блокам
        return x
    
# 3.2 
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class BottleneckBlock(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(BottleneckBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    
class WideResidualBlock(nn.Module):
    #используем увеличенное количество фильтров в каждом слое
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, widen_factor=2):
        super(WideResidualBlock, self).__init__()
        planes = int(planes * widen_factor)

        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * self.expansion)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [70]:
config_3_layer = {
    "input_size": 784,
    "num_classes": 10,
    "layers": [
        {"type": "linear", "size": 512},
        {"type": "relu"},
        {"type": "batch_norm"},

        {"type": "linear", "size": 256},
        {"type": "relu"},
        {"type": "dropout", "rate": 0.4}
    ]
}
config_7_layer = {
    "input_size": 3072,
    "num_classes": 10,
    "layers": [
        {"type": "linear", "size": 512},
        {"type": "relu"},
        {"type": "linear", "size": 256},
        {"type": "relu"},
        {"type": "linear", "size": 128},
        {"type": "relu"},
        {"type": "linear", "size": 64},
        {"type": "relu"},
        {"type": "linear", "size": 32},
        {"type": "relu"},
    ]
}


In [72]:
# Создаем функцию для grad flow 
def get_gradient_flow(model):
    avg_grads = []
    
    for name, param in model.named_parameters():
        if 'weight' in name and param.grad is not None:
            avg_grad = param.grad.abs().mean().item()#Считаем среднее значение градиента по модулю
            avg_grads.append((name, avg_grad))
    
    return avg_grads

In [74]:
def run_epoch(model, data_loader, criterion, optimizer=None, device='cpu', is_test=False):
    if is_test:
        model.eval()
    else:
        model.train()
    
    total_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (data, target) in enumerate(tqdm(data_loader)):
        data, target = data.to(device), target.to(device)
        
        if not is_test and optimizer is not None:
            optimizer.zero_grad()
        
        output = model(data)
        loss = criterion(output, target)
        
        if not is_test and optimizer is not None:
            loss.backward()   
            optimizer.step()
        
        total_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        total += target.size(0)
    
    return total_loss / len(data_loader), correct / total


def train_model(model, train_loader, test_loader, epochs=10, lr=0.001, device='cpu'):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    train_losses, train_accs = [], []
    test_losses, test_accs = [], []
    
    for epoch in range(epochs):
        train_loss, train_acc = run_epoch(model, train_loader, criterion, optimizer, device, is_test=False)
        test_loss, test_acc = run_epoch(model, test_loader, criterion, None, device, is_test=True)
        if epochs % 2 == 0:
            grads = get_gradient_flow(model)# вызов градиентов 
            for name, grad in grads:
                print(f"{name}: {grad:.6f}")
                
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        test_losses.append(test_loss)
        test_accs.append(test_acc)
        
        print(f'Epoch {epoch+1}/{epochs}:')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')
        print('-' * 50)
    
    return {
        'train_losses': train_losses,
        'train_accs': train_accs,
        'test_losses': test_losses,
        'test_accs': test_accs
    } 

In [76]:
def plot_training_history(history):
    """Визуализирует историю обучения"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    ax1.plot(history['train_losses'], label='Train Loss')
    ax1.plot(history['test_losses'], label='Test Loss')
    ax1.set_title('Loss')
    ax1.legend()
    
    ax2.plot(history['train_accs'], label='Train Acc')
    ax2.plot(history['test_accs'], label='Test Acc')
    ax2.set_title('Accuracy')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()


def count_parameters(model):
    """Подсчитывает количество параметров модели"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def save_model(model, path):
    """Сохраняет модель"""
    torch.save(model.state_dict(), path)


def load_model(model, path):
    """Загружает модель"""
    model.load_state_dict(torch.load(path))
    return model


def compare_models(fc_history, cnn_history):
    """Сравнивает результаты полносвязной и сверточной сетей"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    ax1.plot(fc_history['test_accs'], label='FC Network', marker='o')
    ax1.plot(cnn_history['test_accs'], label='CNN', marker='s')
    ax1.set_title('Test Accuracy Comparison')
    ax1.legend()
    ax1.grid(True)
    
    ax2.plot(fc_history['test_losses'], label='FC Network', marker='o')
    ax2.plot(cnn_history['test_losses'], label='CNN', marker='s')
    ax2.set_title('Test Loss Comparison')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show() 

def compute_confusion_matrix(model, data_loader, device='cpu'):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            preds = output.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    cm = confusion_matrix(all_targets, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()
def visualize_first_layer_activations(model, data_loader, device, title="Model"):
    model.eval()
    # Берём первую batch для визуализации
    data, _ = next(iter(data_loader))
    data = data.to(device)
    
    # Получаем выход первого сверточного слоя
    first_conv_layer = None
    for layer in model.children():
        if isinstance(layer, nn.Conv2d):
            first_conv_layer = layer
            break
    
    if first_conv_layer is None:
        print("Нет сверточных слоев в модели.")
        return

    # Forward до первого сверточного слоя
    with torch.no_grad():
        activations = first_conv_layer(data)

    # Берём первую картинку из батча
    img = data[0].cpu().numpy().transpose((1, 2, 0))  # CHW -> HWC
    img = (img - img.min()) / (img.max() - img.min())  # Нормируем

    # Визуализация
    fig, axes = plt.subplots(1, activations.size(1) + 1, figsize=(15, 3))
    axes[0].imshow(img.squeeze(), cmap='gray')
    axes[0].set_title('Input Image')
    axes[0].axis('off')

    for i in range(activations.size(1)):
        act = activations[0, i].cpu().numpy()
        act = (act - act.min()) / (act.max() - act.min())
        axes[i + 1].imshow(act, cmap='viridis')
        axes[i + 1].set_title(f'Ch {i+1}')
        axes[i + 1].axis('off')

    plt.suptitle(f"{title} - First Layer Activations")
    plt.tight_layout()
    plt.show()
    
def compare_kernel_results(results):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Сравнение точности
    for name in results:
        ax1.plot(results[name]['history']['test_accs'], label=name)
    ax1.set_title('Test Accuracy Comparison')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)

    # Сравнение времени обучения
    names = list(results.keys())
    times = [results[name]['time'] / 60 for name in names]  # в минутах
    ax2.bar(names, times)
    ax2.set_title('Training Time (minutes)')
    ax2.set_ylabel('Time (min)')
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

# Задание 3: Кастомные слои и эксперименты

# 3.1 Реализация кастомных слоев
## Кастомный сверточный слой с дополнительной логикой

In [52]:
x = torch.randn(1, 3, 32, 32)  # batch, channels, height, width

custom_conv = CustomConv2d(3, 16, 3, padding=1)
standard_conv = nn.Conv2d(3, 16, 3, padding=1)
y_custom = custom_conv(x)
y_standard = standard_conv(x)

print("Размер кастомного сверточного слоя:", y_custom.shape)
print("Размер стандартного сверточного слоя:", y_standard.shape)
print("Выводы совпадают или нет?:", torch.allclose(y_custom, y_standard))

Размер кастомного сверточного слоя: torch.Size([1, 16, 32, 32])
Размер стандартного сверточного слоя: torch.Size([1, 16, 32, 32])
Выводы совпадают или нет?: False


Положительное:
Архитектура слоя корректна с точки зрения размерностей : кастомный слой возвращает тензор нужной формы.
Отрицательное:
Значения не совпадают, а значит, что мне удалось создать свой сверточный слой, который отличается от стандартного, так как в моем я добавляю также шумы в слое

## Attention механизм

In [61]:
x = torch.randn(1, 3, 32, 32)
attention_layer = SpatialAttention(kernel_size=7)
# Применение слоя
output = attention_layer(x)

print("Размер выхода:", output.shape)
print("Минимальное значение маски:", output.min().item())
print("Максимальное значение маски:", output.max().item())

Размер выхода: torch.Size([1, 1, 32, 32])
Минимальное значение маски: 0.30229032039642334
Максимальное значение маски: 0.8292129635810852


Судя по результатам все работает корректно. 
Размер выхода совпадает с ожидаемым (1, 1, 32, 32), значения маски лежат в диапазоне сигмоиды (от ~0.3 до ~0.8). В отличие от стандартных механизмов, мой фокусируется только на пространственных признаках, что выходит быстрее, но менее информативным, так как не учитывает важность всех каналов.

## Кастомная функция активации

In [82]:
x = torch.randn(5, requires_grad=True)
y_custom = CustomReLU().forward(x)
y_torch = torch.relu(x)

print("Кастомный ReLU", y_custom)
print("Торчевский ReLU", y_torch)
print("Результаты совпадают? ", torch.allclose(y_custom, y_torch))


Кастомный ReLU tensor([0.0000, 0.8280, 0.6459, 0.1727, 0.0000], grad_fn=<ClampBackward1>)
Торчевский ReLU tensor([0.0000, 0.8280, 0.6459, 0.1727, 0.0000], grad_fn=<ReluBackward0>)
Результаты совпадают?  True


Кастомная реализация CustomReLU возвращает те же значения, что и стандартный torch.relu, о чём свидетельствует результат torch.allclose = True. Разница только в методе вычисления градиентов (ClampBackward1 и ReLUBackward0). В целом, по выходу функции моя реализация эквивалентна стандартной.

## Кастомный pooling слой

In [87]:
x = torch.randn(1, 3, 32, 32)

custom_pool = CustomAdaptiveAvgPool2d(output_size=(4, 4))
output_custom = custom_pool(x)

standard_pool = nn.AdaptiveAvgPool2d(output_size=(4, 4))
output_standard = standard_pool(x)

print("Размер выхода кастомного слоя:", output_custom.shape)
print("Размер выхода стандартного слоя:", output_standard.shape)
print("Выводы совпадают или нет?:", torch.allclose(output_custom, output_standard))

Размер выхода кастомного слоя: torch.Size([1, 3, 4, 4])
Размер выхода стандартного слоя: torch.Size([1, 3, 4, 4])
Выводы совпадают или нет?: True


Кастомный pooling слой корректно реализует функционал адаптивного усредняющего пулинга и возвращает выход того же размера что и стандартный. Результаты работы слоёв совпадают по значениям что подтверждает корректность реализации. По функциональности и производительности он эквивалентен стандартному слою но при этом даёт возможность модификации логики, что круто

# 3.2 Эксперименты с Residual блоками

## Базовый Residual блок

In [96]:
x = torch.randn(1, 64, 32, 32)
block = BasicBlock(64, 64)
output = block(x)

print("Размер выхода:", output.shape)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Количество параметров в BasicBlock:", count_parameters(block))

Размер выхода: torch.Size([1, 64, 32, 32])
Количество параметров в BasicBlock: 73984


BasicBlock корректно обработал входной тензор и вернул выход той же размерности (1, 64, 32, 32) что и ожидалось. Количество обучаемых параметров в блоке составило 73984 что соответствует ожиданиям для двух сверточных слоёв с batch нормализацией. Блок стабильно сохраняет размерность карт признаков и имеет разумное количество параметров подходящее для средних сетей. Это делает его устойчивым к переобучению

## Bottleneck Residual блок

In [101]:
x = torch.randn(1, 64, 32, 32)
block = BottleneckBlock(64, 64)
output = block(x)
print("Размер выхода:", output.shape)

Размер выхода: torch.Size([1, 256, 32, 32])


In [103]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Количество параметров в BottleneckBlock:", count_parameters(block))

Количество параметров в BottleneckBlock: 75008


BottleneckBlock корректно обработал входной тензор и увеличил количество каналов с 64 до 256 что соответствует ожидаемому поведению этого типа блока. Количество обучаемых параметров составило 75008 что оказалось меньше чем у BasicBlock в предыдущем задании из-за более эффективной архитектуры с узкими промежуточными слоями. Блок сохранил пространственные размеры изображения и показал стабильное поведение при прямом проходе что делает его подходящим для построения глубоких сетей с residual связями.

## Wide Residual блок

In [113]:
x = torch.randn(1, 64, 32, 32)
block = WideResidualBlock(64, 64, widen_factor=2)
output = block(x)
print("Размер выхода:", output.shape)

Размер выхода: torch.Size([1, 128, 32, 32])


In [117]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Количество параметров в WideResidualBlock:", count_parameters(block))

Количество параметров в WideResidualBlock: 230144


WideResidualBlock корректно обработал входной тензор и увеличил количество каналов с 64 до 128 за счёт параметра widen_factor=2 что соответствует ожидаемой работе широкого residual блока. Количество обучаемых параметров составило 230144 что значительно превышает BasicBlock и даже BottleneckBlock из-за увеличенных слоёв. Это делает WideResidualBlock более ёмким по памяти но потенциально способным к лучшему представлению признаков